In [1]:
import telebot
import mediapipe as mpipe
import math
import cv2
import numpy as np
from time import time
import matplotlib.pyplot as plt
import requests
import speech_recognition as sr
import moviepy.editor as mp
from pydub import AudioSegment
import os

In [2]:
token = "5260396570:AAHj_54xGrUdg0tZrAdUS8VfBbnnUTiqBZA"
bot = telebot.TeleBot(token)
r = sr.Recognizer()
video_folder = os.path.join(os.getcwd(), 'video')

# Initializing mediapipe pose class.
mp_pose = mpipe.solutions.pose

# Setting up the Pose function.
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.3, model_complexity=2)

# Initializing mediapipe drawing class, useful for annotation.
mp_drawing = mpipe.solutions.drawing_utils 

In [3]:
def video_converter(file_name, file):
    if not os.path.exists(video_folder):
        os.makedirs(video_folder)
    path_mp4 = os.path.join(video_folder, f"{file_name}.mp4")
    with open(path_mp4, 'wb') as f:
        f.write(file.content)
    clip = mp.VideoFileClip(path_mp4)
    return path_mp4

In [4]:
def empty_folders():
    for filename in os.listdir(video_folder):
        file_path = os.path.join(video_folder, filename)
        try:
            os.remove(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

In [5]:
def detectPose(image, pose, display=True):
    '''
    This function performs pose detection on an image.
    Args:
        image: The input image with a prominent person whose pose landmarks needs to be detected.
        pose: The pose setup function required to perform the pose detection.
        display: A boolean value that is if set to true the function displays the original input image, the resultant image, 
                 and the pose landmarks in 3D plot and returns nothing.
    Returns:
        output_image: The input image with the detected pose landmarks drawn.
        landmarks: A list of detected landmarks converted into their original scale.
    '''
    
    # Create a copy of the input image.
    output_image = image.copy()
    
    # Convert the image from BGR into RGB format.
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Perform the Pose Detection.
    results = pose.process(imageRGB)
    
    # Retrieve the height and width of the input image.
    height, width, _ = image.shape
    
    # Initialize a list to store the detected landmarks.
    landmarks = []
    
    # Check if any landmarks are detected.
    if results.pose_landmarks:
    
        # Draw Pose landmarks on the output image.
        mp_drawing.draw_landmarks(image=output_image, landmark_list=results.pose_landmarks,
                                  connections=mp_pose.POSE_CONNECTIONS)
        
        # Iterate over the detected landmarks.
        for landmark in results.pose_landmarks.landmark:
            
            # Append the landmark into the list.
            landmarks.append((int(landmark.x * width), int(landmark.y * height),
                                  (landmark.z * width)))
    
    # Check if the original input image and the resultant image are specified to be displayed.
    if display:
    
        # Display the original input image and the resultant image.
        plt.figure(figsize=[22,22])
        plt.subplot(121);plt.imshow(image[:,:,::-1]);plt.title("Original Image");plt.axis('off');
        plt.subplot(122);plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');
        
        # Also Plot the Pose landmarks in 3D.
        mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
        
    # Otherwise
    else:
        
        # Return the output image and the found landmarks.
        return output_image, landmarks

In [6]:
def calculateAngle(landmark1, landmark2, landmark3):
    '''
    This function calculates angle between three different landmarks.
    Args:
        landmark1: The first landmark containing the x,y and z coordinates.
        landmark2: The second landmark containing the x,y and z coordinates.
        landmark3: The third landmark containing the x,y and z coordinates.
    Returns:
        angle: The calculated angle between the three landmarks.

    '''

    # Get the required landmarks coordinates.
    x1, y1, _ = landmark1
    x2, y2, _ = landmark2
    x3, y3, _ = landmark3

    # Calculate the angle between the three points
    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
    
    # Check if the angle is less than zero.
    if angle < 0:

        # Add 360 to the found angle.
        angle += 360
    
    # Return the calculated angle.
    return angle

In [7]:
def classifyPose(landmarks, output_image, display=False):
    # Initialize the label of the pose. It is not known at this stage.
    label = 'Unknown Pose'
    color = (0, 0, 255)
    
    # Get the angle between the left shoulder, elbow and wrist points. 
    left_elbow_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value])
    
    # Get the angle between the right shoulder, elbow and wrist points. 
    right_elbow_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value])   
    
    # Get the angle between the left shoulder, knee and hip points. 
    left_shoulder_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                         landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                         landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value])

    # Get the angle between the right hip, shoulder, knee  
    right_shoulder_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value])

    # Get the angle between the left hip, knee and ankle points. 
    left_knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])

    # Get the angle between the right hip, knee and ankle points 
    right_knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                      landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])
    
    
    # Check if both legs are straight
    if left_knee_angle > 160 and left_knee_angle < 195 and right_knee_angle > 160 and right_knee_angle < 195:

        # Check hip-shoulder angle
        if left_shoulder_angle > 120 and left_shoulder_angle < 195 and right_shoulder_angle > 160 and right_shoulder_angle < 195:     
            label = 'Staying'

    
    # Check if the pose is classified successfully
    if label != 'Unknown Pose':
        
        # Update the color (to green) with which the label will be written on the image.
        color = (0, 255, 0)  
    
    # Write the label on the output image. 
    cv2.putText(output_image, label, (10, 80),cv2.FONT_HERSHEY_PLAIN, 7, color, 2)
    
    # Check if the resultant image is specified to be displayed.
#     if display:
    
#         # Display the resultant image.
#         plt.figure(figsize=[10,10])
#         plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');
        
#     else:
        
#         # Return the output image and the classified label.
#         return output_image, label
    return label

In [8]:
def pose_response(file_name):
    cap = cv2.VideoCapture('video/' + file_name + '.mp4')
    text = 'Что-то пошло не так('
    while True:
        ret, frame = cap.read()
        if ret:
            output_image, landmarks = detectPose(frame, pose, display=False)
            if landmarks:
                text = classifyPose(landmarks, output_image, display=True)
                if text != 'Unknown Pose' and text != 'Что-то пошло не так(':
                    break;

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

    return text

In [9]:
@bot.message_handler(commands=["start", "help"])
def start_processing(message):
    text = ("Hello world)))\n")
    bot.reply_to(message, text)


@bot.message_handler(content_types=['video_note'])
def videomessage_processing(message):
    file_info = bot.get_file(message.video_note.file_id)
    file_name = file_info.file_path.split('/')[1]
    file = requests.get('https://api.telegram.org/file/bot{0}/{1}'.format(token, file_info.file_path))
    video = video_converter(file_name, file)
    response = pose_response(file_name)
    empty_folders()
    text = 'Не понимаю что происходит('
    if response == 'Staying':
        text = 'Говорят, в ногах правды нет...'
    if response == 'Unknown Pose':
        text = 'Что Вы делаете?)'
    bot.reply_to(message, text)

In [ ]:
if __name__ == '__main__':
    while True:
        try:
            bot.polling(2)
        except Exception as error:
            print(error)